In [1]:
#@title DPO 방식으로 모델 재학습 (A100 최적화 및 CUDA 오류 해결 최종판 - 2025 업데이트)

# --- 1. ★ 최종 라이브러리 설치 ---
# 2025년 기준 최신 버전 + bitsandbytes 호환 설치 (CUDA 12.6 지원)
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub bitsandbytes
!pip install -q "pandas==2.2.2"

In [2]:
# --- 2. 파일 업로드 및 로그인 ---
from google.colab import files
from huggingface_hub import login
import os

# 파일 업로드 (기존 파일이 있으면 건너뜁니다)
files_to_upload = ['dpo_dataset.jsonl', 'styles.csv']
for filename in files_to_upload:
    if not os.path.exists(filename):
        print(f"'{filename}'을 업로드해주세요.")
        files.upload()
    else:
        print(f"'{filename}'이(가) 이미 존재합니다.")

# Hugging Face 로그인
print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
login()
print("✅ 로그인 성공!")

'dpo_dataset.jsonl'이(가) 이미 존재합니다.
'styles.csv'이(가) 이미 존재합니다.

--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [3]:
# --- 3. DPO 학습 (A100 최적화 최종 코드) ---
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig
from trl import DPOTrainer, DPOConfig  # DPOConfig 임포트

# 환경 변수 설정 (CUDA 라이브러리 경로 추가: bitsandbytes 오류 방지)
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:/usr/lib64-nvidia:' + os.environ.get('LD_LIBRARY_PATH', '')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # GPU 강제 지정

# GPU 확인 (디버깅용: True여야 함)
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")
if not torch.cuda.is_available():
    raise RuntimeError("GPU 런타임이 설정되지 않았습니다. Runtime > Change runtime type > GPU 선택 후 재시작하세요.")

# 메모리 클리어
torch.cuda.empty_cache()

model_id = "google/gemma-2b-it"
new_model_name = "gemma-fashion-dpo-final-v3"

# 모델 로드 (load_in_8bit=False로 bitsandbytes 우회)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,  # 메모리 최적화
    load_in_8bit=False,  # 8-bit 양자화 비활성 (bitsandbytes 호출 방지)
)

# ref_model 별도 로드 (메모리 소모 증가지만 안정)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
    load_in_8bit=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 로드 및 검증
dataset = load_dataset("json", data_files="dpo_dataset.jsonl", split="train")
required_columns = {'prompt', 'chosen', 'rejected'}
if not required_columns.issubset(dataset.column_names):
    missing = required_columns - set(dataset.column_names)
    raise ValueError(f"데이터셋에 누락된 컬럼: {missing}.")

# 작은 서브셋 테스트 (디버깅용; 주석 해제)
# dataset = dataset.select(range(10))

print(f"✅ 데이터셋 로드 완료: {len(dataset)} 샘플")

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# DPO 학습 설정 (force_use_ref_model=True를 DPOConfig에 추가: ValueError 방지)
training_args = DPOConfig(
    output_dir=f"./{new_model_name}",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
    logging_steps=25,
    save_steps=100,
    bf16=True,
    remove_unused_columns=False,
    report_to="none",
    max_prompt_length=128,
    max_length=256,
    disable_dropout=True,  # 안정화 (드롭아웃 비활성)
    force_use_ref_model=True,  # 핵심 수정: ref_model + peft_config 허용 (DPOConfig에 위치)
)

# DPOTrainer 생성 (force_use_ref_model 제거: DPOTrainer가 아닌 DPOConfig에 속함)
dpo_trainer = DPOTrainer(
    model,
    ref_model=ref_model,  # 명시적 ref_model 유지
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
)

print("\n🚀 DPO 방식으로 모델 Fine-tuning을 시작합니다...")

try:
    dpo_trainer.train()
    print("✅ DPO 모델 Fine-tuning 완료!")
except RuntimeError as e:
    if "out of memory" in str(e):
        print("❌ OOM 오류! 배치 크기 줄이세요.")
    elif "CUDA" in str(e):
        print("❌ CUDA 오류 지속! bitsandbytes 소스 컴파일 시도: 아래 명령 실행.")
        print("!git clone https://github.com/bitsandbytes-foundation/bitsandbytes.git")
        print("!cd bitsandbytes && make CUDA_VERSION=126 && python setup.py install")
    else:
        raise e



GPU 사용 가능: True
CUDA 버전: 12.6


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 데이터셋 로드 완료: 7699 샘플


...

Applying chat template to train dataset:   0%|          | 0/7699 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7699 [00:00<?, ? examples/s]


🚀 DPO 방식으로 모델 Fine-tuning을 시작합니다...


Step,Training Loss
25,0.682900
50,0.655000
75,0.619900
100,0.595700
125,0.581400
150,0.559900
175,0.555400
200,0.541500
225,0.536200
250,0.508000


✅ DPO 모델 Fine-tuning 완료!


In [4]:
# --- 4. 모델 저장 및 다운로드 ---
print(f"\n학습된 모델을 '{new_model_name}'에 저장합니다...")
dpo_trainer.save_model(f"./{new_model_name}")
print("압축을 시작합니다...")
!zip -r /content/{new_model_name}.zip /content/{new_model_name}
print("✅ 압축 완료! 다운로드를 시작합니다.")
files.download(f'/content/{new_model_name}.zip')


학습된 모델을 'gemma-fashion-dpo-final-v3'에 저장합니다...
압축을 시작합니다...
  adding: content/gemma-fashion-dpo-final-v3/ (stored 0%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/ (stored 0%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/optimizer.pt (deflated 8%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/README.md (deflated 65%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/rng_state.pth (deflated 26%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/adapter_config.json (deflated 57%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/tokenizer.json (deflated 84%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/tokenizer.model (deflated 51%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/special_tokens_map.json (deflated 70%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/chat_template.jinja (deflated 52%)
  adding: content/gemma-fashion-dpo-final-v3/checkpoint-200/scheduler.pt (def

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>